In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression 
import matplotlib.pylab as pylab
import math,time
import sklearn
import sklearn.preprocessing
import datetime
import os
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout, Activation,Flatten
from sklearn.preprocessing import MinMaxScaler
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from keras.layers import Bidirectional
test_set_size_percentage = 20 
import pandas as pd
from pandas import read_csv
from sklearn.metrics import r2_score,mean_squared_error 

Using TensorFlow backend.


In [3]:
data=pd.read_csv('dataset/iron_oilmonthly.csv',parse_dates=['date'])
data=data.dropna()
#df=data[['oildata','date']]
data=data.set_index('date')
data=data.dropna()
data.head()

,ironprice,oildata
date,,
2010-10-31,148.48,79.662
2010-11-30,160.55,79.662
2010-12-31,168.53,79.662
2011-01-31,179.63,79.662
2011-02-28,187.18,79.662


In [4]:
data['oildata_lag']=data['oildata'].shift(1)
#data=data.set_index('date')
data['rolling_mean_price']=data['oildata_lag'].rolling(2, min_periods=1).sum()
data=data.dropna()

In [5]:
# function for min-max normalization of Iron Ore 
min_max_scaler = sklearn.preprocessing.MinMaxScaler()
def normalize_data(df):
    df['oildata'] = min_max_scaler.fit_transform(df.oildata.values.reshape(-1,1))
    df['oildata_lag'] = min_max_scaler.fit_transform(df.oildata_lag.values.reshape(-1,1))
    df['ironprice'] = min_max_scaler.fit_transform(df.ironprice.values.reshape(-1,1))
    df['rolling_mean_price'] = min_max_scaler.fit_transform(df.rolling_mean_price.values.reshape(-1,1))
    return df

In [6]:
# normalize stock
df = data.copy()
df = normalize_data(df)
df.head()

,ironprice,oildata,oildata_lag,rolling_mean_price
date,,,,
2010-11-30,0.817977,0.551036,0.551036,0.131441
2010-12-31,0.872522,0.551036,0.551036,0.559881
2011-01-31,0.948394,0.551036,0.551036,0.559881
2011-02-28,1.000000,0.551036,0.551036,0.559881
2011-03-31,0.878195,0.551036,0.551036,0.559881


In [7]:
train,test=df[:-5],df[-5:]

In [8]:
train.tail()

,ironprice,oildata,oildata_lag,rolling_mean_price
date,,,,
2018-08-31,0.185509,0.383269,0.418256,0.396333
2018-09-30,0.190841,0.386173,0.383269,0.403946
2018-10-31,0.212850,0.434576,0.386173,0.387300
2018-11-30,0.214764,0.493516,0.434576,0.413920
2018-12-31,0.191524,0.484264,0.493516,0.469614


In [9]:
test.head()

,ironprice,oildata,oildata_lag,rolling_mean_price
date,,,,
2019-01-31,0.193575,0.484739,0.484264,0.495393
2019-02-28,0.241422,0.474280,0.484739,0.490840
2019-03-31,0.323445,0.525297,0.474280,0.485660
2019-04-30,0.311825,0.548185,0.525297,0.506703
2019-05-31,0.361039,0.402495,0.548185,0.545047


In [10]:
print(len(train), len(test))

98 5


In [11]:
train_data_raw = train.as_matrix()
test_data_raw = test.as_matrix()

C:\Users\sumon\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\sumon\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [12]:
print("train data shape = ", train_data_raw.shape)
print("test data shape = ",test_data_raw.shape)

train data shape =  (98, 4)
test data shape =  (5, 4)


In [13]:
x_train = train_data_raw[:,1:] #without Iron price column of train
y_train = train_data_raw[:,0] #Only Iron price column of train

x_test = test_data_raw[:,1:] #without Iron price column of test
y_test = test_data_raw[:,0]  #Only Iron price column of test

In [14]:
x_train[0:5]

array([[0.55103581, 0.55103581, 0.13144078],
       [0.55103581, 0.55103581, 0.55988062],
       [0.55103581, 0.55103581, 0.55988062],
       [0.55103581, 0.55103581, 0.55988062],
       [0.55103581, 0.55103581, 0.55988062]])

In [15]:
y_train[0:5]

array([0.81797676, 0.87252221, 0.94839371, 1.        , 0.87819549])

In [16]:
train_data_raw[0:5]

array([[0.81797676, 0.55103581, 0.55103581, 0.13144078],
       [0.87252221, 0.55103581, 0.55103581, 0.55988062],
       [0.94839371, 0.55103581, 0.55103581, 0.55988062],
       [1.        , 0.55103581, 0.55103581, 0.55988062],
       [0.87819549, 0.55103581, 0.55103581, 0.55988062]])

In [17]:
# def denormalize(df, normalized_value): 
#     df = df['Price'].values.reshape(-1,1)
#     print(df[:5])
#     normalized_value = normalized_value.reshape(-1,1)
#     #return df.shape, p.shape
#     print(normalized_value[0:5])
#     min_max_scaler = sklearn.preprocessing.MinMaxScaler()
#     a = min_max_scaler.fit_transform(df)
#     print(a[:5])
#     new = min_max_scaler.inverse_transform(normalized_value)
#     print(new[:10])
#     return new

# #newp = denormalize(test_value_store, pred_price)
# newy_test = denormalize(data, y_test)

In [18]:
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ', y_train.shape)
print('x_test.shape = ', x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train.shape =  (98, 3)
y_train.shape =  (98,)
x_test.shape =  (5, 3)
y_test.shape =  (5,)


In [19]:
#Reshape data for (Sample,Timestep,Features) 
x_train = x_train.reshape((x_train.shape[0],1,x_train.shape[1]))
x_test = x_test.reshape((x_test.shape[0],1,x_test.shape[1]))
#x_forecast = x_forecast.reshape((x_forecast.shape[0],1,x_test.shape[1]))
x_test.shape

(5, 1, 3)

In [20]:
(train_dim, train_timestep , train_feat) = x_train.shape
(test_dim, test_timestep , test_feat) = x_test.shape

In [21]:
from keras import optimizers
print("Build Model")
"""
model = Sequential()

model.add(LSTM(16, input_shape=(train_timestep,train_feat), return_sequences=True))
#model.add(Dropout(0.3))

model.add(LSTM(16, input_shape=(train_timestep,train_feat), return_sequences=False))
#model.add(Dropout(0.3))

model.add(Dense(8,kernel_initializer="uniform",activation='relu'))        
model.add(Dense(1,kernel_initializer="uniform",activation='linear'))

# adam = keras.optimizers.Adam(decay=0.2)

start = time.time()
model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
print("Compilation Time : ", time.time() - start)
"""
model = Sequential()

model.add(Bidirectional(LSTM(16, input_shape=(train_timestep,train_feat), return_sequences=True)))
#model.add(Dropout(0.3))

model.add(Dense(8,kernel_initializer="uniform",activation='relu'))  
model.add(Flatten())
model.add(Dense(1,kernel_initializer="uniform",activation='linear'))

#adam = keras.optimizers.Adam(decay=0.2)
#sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
start = time.time()
model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
#model.compile(loss='mse',metrics=['accuracy'],optimizer=adam)
print("Compilation Time : ", time.time() - start)


Build Model
Compilation Time :  0.018949270248413086


In [22]:
#Fit model with history to check for overfitting
history = model.fit(x_train,y_train,batch_size = 1,epochs=20,shuffle=True)


Epoch 1/20
98/98 [==============================] - 1s 7ms/step - loss: 0.1743 - accuracy: 0.0102
Epoch 2/20
98/98 [==============================] - 0s 1ms/step - loss: 0.0575 - accuracy: 0.0102
Epoch 3/20
98/98 [==============================] - 0s 1ms/step - loss: 0.0551 - accuracy: 0.0102
Epoch 4/20
98/98 [==============================] - 0s 2ms/step - loss: 0.0552 - accuracy: 0.0102
Epoch 5/20
98/98 [==============================] - 0s 1ms/step - loss: 0.0536 - accuracy: 0.0102
Epoch 6/20
98/98 [==============================] - 0s 2ms/step - loss: 0.0528 - accuracy: 0.0102
Epoch 7/20
98/98 [==============================] - 0s 2ms/step - loss: 0.0519 - accuracy: 0.0102
Epoch 8/20
98/98 [==============================] - 0s 2ms/step - loss: 0.0520 - accuracy: 0.0102
Epoch 9/20
98/98 [==============================] - 0s 1ms/step - loss: 0.0518 - accuracy: 0.0102
Epoch 10/20
98/98 [==============================] - 0s 2ms/step - loss: 0.0531 - accuracy: 0.0102
Epoch 11/20
98/98 

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 1, 32)             2560      
_________________________________________________________________
dense_1 (Dense)              (None, 1, 8)              264       
_________________________________________________________________
flatten_1 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 2,833
Trainable params: 2,833
Non-trainable params: 0
_________________________________________________________________


In [24]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [25]:
# load json and create model
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])

Loaded model from disk


In [26]:
pred_price = model.predict(x_test,batch_size=1,verbose=1)

5/5 [==============================] - 0s 27ms/step


In [27]:
pred_price

array([[0.35505736],
       [0.35197443],
       [0.36154446],
       [0.3797058 ],
       [0.35433984]], dtype=float32)

In [28]:
def denormalize(df, normalized_value): 
    df = df['ironprice'].values.reshape(-1,1)
    normalized_value = normalized_value.reshape(-1,1)
    #return df.shape, p.shape
    min_max_scaler = sklearn.preprocessing.MinMaxScaler()
    a = min_max_scaler.fit_transform(df)
    new = min_max_scaler.inverse_transform(normalized_value)
    return new

newp = denormalize(data, pred_price)
newy_test = denormalize(data, y_test)

In [29]:
newp

array([[92.8249  ],
       [92.37386 ],
       [93.77396 ],
       [96.43096 ],
       [92.719925]], dtype=float32)

In [30]:
newy_test

array([[69.2],
       [76.2],
       [88.2],
       [86.5],
       [93.7]])

In [31]:
pred_oil_pirce = pd.concat([pd.DataFrame(newp),pd.DataFrame(newy_test)], axis=1) 
pred_oil_pirce.columns = ['Pred_Oil_Price','Actual_Oil_Price']
pred_oil_pirce['date'] = test.index
pred_oil_pirce = pred_oil_pirce.set_index('date')

In [32]:
pred_oil_pirce

,Pred_Oil_Price,Actual_Oil_Price
date,,
2019-01-31,92.824898,69.2
2019-02-28,92.373863,76.2
2019-03-31,93.773956,88.2
2019-04-30,96.430962,86.5
2019-05-31,92.719925,93.7


In [33]:
def model_score(model, x_train, y_train, x_test, y_test):
    trainScore = model.evaluate(x_train, y_train, verbose=0)
    print('Train Score: %.5f MSE (%.2f RMSE)' % (trainScore[0], math.sqrt(trainScore[0])))

    testScore = model.evaluate(x_test, y_test, verbose=0)
    print('Test Score: %.5f MSE (%.2f RMSE)' % (testScore[0], math.sqrt(testScore[0])))
    return trainScore[0], testScore[0]


model_score(model, x_train, y_train, x_test, y_test)

Train Score: 0.04964 MSE (0.22 RMSE)
Test Score: 0.00888 MSE (0.09 RMSE)


(0.049635006913117, 0.008880549110472202)

In [34]:
def pred_accuracy(pred, actual):
    mape=np.mean(np.abs(pred-actual)/np.abs(actual))# MAPE
    me=np.mean(pred-actual)# ME
    mae=np.mean(np.abs(pred-actual))# MAE
    mpe=np.mean((pred-actual)/actual)# MPE
    return({'mape':mape,'me':me,'mae': mae,'mpe': mpe})
print(pred_accuracy(newp, newy_test))

{'mape': 0.1484241960723261, 'me': 10.864720764160154, 'mae': 11.25675079345703, 'mpe': 0.1442403110211321}


In [37]:
forecast_df = pd.read_csv('dataset/oil_forecast_data.csv')
print(forecast_df)
forecast_df = forecast_df[['date','sarima_oilprice']]
forecast_df = forecast_df.set_index('date')


          date  sarima_oilprice  arima_oilprice
0   2019-06-30        56.893623       75.921675
1   2019-07-31        57.894870       76.545122
2   2019-08-31        57.980675       76.124720
3   2019-09-30        62.527335       76.425577
4   2019-10-31        67.823088       76.223143
5   2019-11-30        65.232596       76.580996
6   2019-12-31        61.762357       76.418885
7   2020-01-31        64.043057       76.765533
8   2020-02-29        64.696372       76.615897
9   2020-03-31        71.018727       76.942060
10  2020-04-30        75.920763       76.800326
11  2020-05-31        67.395489       77.105789
12  2020-06-30        61.538606       76.971002
13  2020-07-31        64.361616       77.256876
14  2020-08-31        65.385406       77.128649
15  2020-09-30        71.921731       77.396177
16  2020-10-31        77.967335       77.274218
17  2020-11-30        72.595228       77.524589
18  2020-12-31        68.064069       77.408629
19  2021-01-31        71.607716       77

In [38]:
#data['Price'].plot() 
forecast_df['oildata_lag']=forecast_df['sarima_oilprice'].shift(1)
#data=data.set_index('date')
forecast_df['rolling_mean_price']=forecast_df['oildata_lag'].rolling(2, min_periods=1).sum()
forecast_df=forecast_df.dropna()
forecast_df

,sarima_oilprice,oildata_lag,rolling_mean_price
date,,,
2019-07-31,57.894870,56.893623,56.893623
2019-08-31,57.980675,57.894870,114.788493
2019-09-30,62.527335,57.980675,115.875545
2019-10-31,67.823088,62.527335,120.508011
2019-11-30,65.232596,67.823088,130.350423
2019-12-31,61.762357,65.232596,133.055684
2020-01-31,64.043057,61.762357,126.994953
2020-02-29,64.696372,64.043057,125.805414
2020-03-31,71.018727,64.696372,128.739429


In [39]:
# function for min-max normalization of Iron Ore 
min_max_scaler = sklearn.preprocessing.MinMaxScaler()
def normalize_data(df):
    df['sarima_oilprice'] = min_max_scaler.fit_transform(df.sarima_oilprice.values.reshape(-1,1))
    df['oildata_lag'] = min_max_scaler.fit_transform(df.oildata_lag.values.reshape(-1,1))
    df['rolling_mean_price'] = min_max_scaler.fit_transform(df.rolling_mean_price.values.reshape(-1,1))
    return df

In [40]:
# normalize stock
df = forecast_df.copy()
df = normalize_data(df)
df.head()

,sarima_oilprice,oildata_lag,rolling_mean_price
date,,,
2019-07-31,0.000000,0.000000,0.000000
2019-08-31,0.002992,0.033739,0.526198
2019-09-30,0.161551,0.036630,0.536078
2019-10-31,0.346232,0.189839,0.578182
2019-11-30,0.255893,0.368290,0.667638


In [41]:
forecast_data_raw = df.as_matrix()

C:\Users\sumon\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [42]:
forecast_data_raw.shape

(23, 3)

In [43]:
forecast_data_raw = forecast_data_raw.reshape((forecast_data_raw.shape[0],1,forecast_data_raw.shape[1]))
forecast_data_raw.shape

(23, 1, 3)

In [44]:
forecast_price = model.predict(forecast_data_raw,batch_size=1,verbose=1)

23/23 [==============================] - 0s 693us/step


In [45]:
forecast_price

array([[0.08248604],
       [0.16181418],
       [0.19906029],
       [0.27738023],
       [0.30278656],
       [0.2612972 ],
       [0.24992019],
       [0.26835424],
       [0.32877615],
       [0.419029  ],
       [0.3919477 ],
       [0.28613487],
       [0.25359014],
       [0.27618423],
       [0.34208843],
       [0.44360763],
       [0.45097432],
       [0.38163936],
       [0.37056717],
       [0.4026068 ],
       [0.4780264 ],
       [0.5808451 ],
       [0.57259613]], dtype=float32)

In [46]:
def denormalize(df, normalized_value): 
    df = df['ironprice'].values.reshape(-1,1)
    normalized_value = normalized_value.reshape(-1,1)
    #return df.shape, p.shape
    min_max_scaler = sklearn.preprocessing.MinMaxScaler()
    a = min_max_scaler.fit_transform(df)
    new = min_max_scaler.inverse_transform(normalized_value)
    return new

newp = denormalize(data, forecast_price)
#newy_test = denormalize(data, y_test)

In [47]:
forecast_df.head()

,sarima_oilprice,oildata_lag,rolling_mean_price
date,,,
2019-07-31,57.894870,56.893623,56.893623
2019-08-31,57.980675,57.894870,114.788493
2019-09-30,62.527335,57.980675,115.875545
2019-10-31,67.823088,62.527335,120.508011
2019-11-30,65.232596,67.823088,130.350423


In [48]:
newp

array([[ 52.94771 ],
       [ 64.55341 ],
       [ 70.00252 ],
       [ 81.46073 ],
       [ 85.17767 ],
       [ 79.10778 ],
       [ 77.44333 ],
       [ 80.14023 ],
       [ 88.97995 ],
       [102.183945],
       [ 98.22195 ],
       [ 82.74153 ],
       [ 77.98023 ],
       [ 81.28575 ],
       [ 90.927536],
       [105.7798  ],
       [106.857544],
       [ 96.713844],
       [ 95.09398 ],
       [ 99.78138 ],
       [110.81526 ],
       [125.85764 ],
       [124.65082 ]], dtype=float32)

In [49]:
act_pred = pd.concat([pd.DataFrame(forecast_df.sarima_oilprice.values),pd.DataFrame(newp)], axis=1) 
act_pred.columns = ['Forecast_Oil_Price','Forecast_Iron_Price']

In [50]:
act_pred['date'] = forecast_df.index

In [51]:
act_pred = act_pred.set_index('date')
act_pred

,Forecast_Oil_Price,Forecast_Iron_Price
date,,
2019-07-31,57.894870,52.947708
2019-08-31,57.980675,64.553413
2019-09-30,62.527335,70.002518
2019-10-31,67.823088,81.460732
2019-11-30,65.232596,85.177673
2019-12-31,61.762357,79.107780
2020-01-31,64.043057,77.443329
2020-02-29,64.696372,80.140228
2020-03-31,71.018727,88.979950


In [49]:
#test_value_store